In [1]:
import khmer
import tqdm
import pandas as pd
# ref = https://github.com/dib-lab/khmer/blob/master/examples/python-api/exact-counting.py

In [ ]:
input_fasta = '/home/ubuntu/data/longread/R2C2_full_length_consensus_reads.fasta'

master_df = pd.DataFrame()
with open(input_fasta, 'r') as f:
    for count, line in tqdm.tqdm(enumerate(f, start=1)):
        if count % 2 != 0:
            k_name = line
        if count % 2 == 0:
            # Initialize countgraph
            ksize = 7
            nkmers = 4**ksize
            tablesize = nkmers + 10
            ktable = khmer.Countgraph(ksize, tablesize, 1)
            
            # count all k-mers in the given string
            ktable.consume(line)

            # run through all entries. if they have nonzero presence, print.
            k_str_list = []
            k_n_list = []
            for i in range(nkmers):
                if ktable.get(i):
                    k_str = ktable.reverse_hash(i)
                    k_n = ktable.get(i)
                    k_str_list = k_str_list+[k_str]
                    k_n_list = k_n_list+[k_n]
            slice_df = pd.DataFrame({k_name:k_n_list,
                                     'k_str':k_str_list,
                                    })
            if len(master_df) == 0:
                master_df = master_df.append(slice_df)
            else:
                master_df = pd.merge(master_df, slice_df, 'left', 'k_str')


1272it [00:56, 11.55it/s]

In [15]:
master_df

,>0bd8b2ae-1105-450d-b548-1d88f64d4a48_9.39_10279_5_2114_1685\n,k_str,>b3ab548f-d732-43b1-a90e-bb30539a6282_9.79_4142_2_1586_1234\n,>9912db26-b531-4e85-be06-aeba22369a22_11.47_12403_11_1236_848\n,>f528f8b0-a0e3-4021-a885-dbc20094f62c_10.26_11614_6_1755_1345\n,>c16e8281-cfb2-46a1-8a42-849e0a420f2e_12.67_4610_2_1733_1335\n,>230ea67c-b659-446a-9712-1e84863a7d8c_10.39_8265_7_1145_734\n,>25aa54f6-838f-40b3-99a6-6a66ebbca3da_12.86_13458_13_977_586\n,>fc354509-456f-4e9b-8306-ac8ecdf30252_11.96_13372_9_1682_1282\n,>a5cc3c55-3622-4ff5-8ccb-676e103ede94_12.01_13468_10_1348_918\n,...,>4deb556d-3abb-4de9-8eb7-d3f3c8ee5f4c_13.21_4197_1_1642_1279\n,>68c7b50e-0d36-442b-8d40-8d368e51c695_9.83_16293_14_1120_727\n,>c588b56b-78f8-4df6-ad64-62989de0da0f_11.36_9785_6_1177_788\n,>435e627a-a791-4b86-a1e4-e3110fc13e33_11.15_10334_7_1376_980\n,>7b5cdee7-9657-452a-a6d0-79992ced0423_10.93_5041_2_2054_1658\n,>1de3e412-9674-44d3-bd07-7e25d61d555b_11.98_4412_3_1113_719\n,>59be8187-f679-499d-ba2f-cfbe6cfcdeaf_12.25_9087_5_1609_1218\n,>def20feb-d973-4527-8a04-192cbf9ce250_13.24_4016_4_752_353\n,>830619b3-7c18-4c12-9efe-b44b09f3ef03_12.12_15800_9_1674_1278\n,>c94102b3-0ed4-4e19-bd7a-bbc2e0053396_11.24_5611_5_954_543\n
0,17,AAAAAAA,20.0,16.0,13.0,1.0,26.0,19.0,18.0,19.0,...,25.0,12.0,5.0,13.0,11.0,14.0,6.0,NaN,19.0,1.0
1,1,AAAAAAG,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,1.0,NaN,3.0,2.0,1.0,1.0,1.0,NaN,1.0,2.0
2,1,AAAAATT,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,NaN
3,1,AAAAACA,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN
4,2,AAAAAGT,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,...,3.0,NaN,1.0,1.0,NaN,1.0,1.0,NaN,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1406,2,GGCTCCC,2.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
1407,3,GGCCCCC,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1408,1,GGCCGCC,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0
1409,3,GGGCCCC,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
